In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [ ]:
"""
This script is for the export and import of all database models as individual files.
This avoids a crashing due to a large db.
"""

In [ ]:
import json
import datetime
from django.apps import apps
from django.core import serializers
from django.conf import settings
from django.db import transaction, IntegrityError

In [ ]:
for model in apps.get_models():
    print(f"{model._meta.app_label}.{model.__name__}")

admin.LogEntry
auth.Permission
auth.Group
auth.User
contenttypes.ContentType
sessions.Session
gis.PostGISGeometryColumns
gis.PostGISSpatialRefSys
swn.ProjectRegion
swn.UserField
swn.SwnProject
swn.NUTS0000_N0
swn.NUTS5000_N1
swn.NUTS5000_N2
swn.NUTS5000_N3
toolbox.ToolboxType
toolbox.AboveGroundWaters
toolbox.BelowGroundWaters
toolbox.UserField
toolbox.SinkWeighting
toolbox.ToolboxProject
toolbox.InfiltrationProject
toolbox.OutlineInjection
toolbox.OutlineSurfaceWater
toolbox.OutlineInfiltration
toolbox.Landuse
toolbox.LanduseMap
toolbox.Stream
toolbox.Lake
toolbox.Sink
toolbox.EnlargedSink
toolbox.Aquifer
toolbox.LanduseSink
toolbox.LanduseEnlargedSink
toolbox.Feasibility
toolbox.Hydrogeology
toolbox.HydrogeologySinks
toolbox.HydrogeologyEnlargedSinks
toolbox.EnlargedSinkEmbankment
toolbox.GroundWaterDistanceClass
toolbox.FieldCapacity
toolbox.Hydromorphy
toolbox.SoilTexture
toolbox.AgriculturalLanduse
toolbox.WetGrassland
toolbox.SoilProperties
toolbox.SinkSoilProperties
toolbox.Enla

In [11]:
def export_all_models(export_dir=None, use_today=True, indent=2):
    # Step 2: Create output directory
    if export_dir is None:
        date_label = datetime.date.today().isoformat() if use_today else "dump"
        export_dir = os.path.join("model_exports", date_label)
    os.makedirs(export_dir, exist_ok=True)
    print(f"📁 Exporting to folder: {export_dir}")

    # Step 3: Loop through all models
    for model in apps.get_models():
        app_label = model._meta.app_label
        model_name = model.__name__
        filename = f"{app_label}.{model_name}.json"
        filepath = os.path.join(export_dir, filename)

        queryset = model.objects.all().order_by(model._meta.pk.name)
        if queryset.exists():
            print(f"📝 Exporting {app_label}.{model_name} ({queryset.count()} rows)...")
            with open(filepath, "w", encoding="utf-8") as f:
                serializers.serialize("json", queryset, indent=indent, stream=f)
        else:
            print(f"⚠️  Skipping {app_label}.{model_name} (no data)")

    print("✅ Done exporting all models.")


In [12]:
export_all_models()

📁 Exporting to folder: model_exports/2025-08-08
📝 Exporting admin.LogEntry (90 rows)...
📝 Exporting auth.Permission (1076 rows)...
⚠️  Skipping auth.Group (no data)
📝 Exporting auth.User (12 rows)...
📝 Exporting contenttypes.ContentType (225 rows)...
📝 Exporting sessions.Session (62 rows)...
📝 Exporting gis.PostGISGeometryColumns (109 rows)...
📝 Exporting gis.PostGISSpatialRefSys (8500 rows)...
📝 Exporting swn.ProjectRegion (1 rows)...
📝 Exporting swn.UserField (28 rows)...
📝 Exporting swn.SwnProject (19 rows)...
⚠️  Skipping swn.NUTS0000_N0 (no data)
📝 Exporting swn.NUTS5000_N1 (16 rows)...
📝 Exporting swn.NUTS5000_N2 (38 rows)...
📝 Exporting swn.NUTS5000_N3 (401 rows)...
📝 Exporting toolbox.ToolboxType (3 rows)...
📝 Exporting toolbox.AboveGroundWaters (74 rows)...
📝 Exporting toolbox.BelowGroundWaters (9006 rows)...
📝 Exporting toolbox.UserField (19 rows)...
⚠️  Skipping toolbox.SinkWeighting (no data)
⚠️  Skipping toolbox.ToolboxProject (no data)
⚠️  Skipping toolbox.InfiltrationPro

In [ ]:


def import_all_models(import_dir):
    print(f"📥 Importing models from: {import_dir}")
    files = sorted(f for f in os.listdir(import_dir) if f.endswith(".json"))

    for file in files:
        path = os.path.join(import_dir, file)
        model_label = file.replace(".json", "")
        print(f"📄 Loading {model_label} from {file}")

        try:
            with open(path, "r", encoding="utf-8") as f:
                objects = list(serializers.deserialize("json", f))
                with transaction.atomic():
                    for obj in objects:
                        obj.save()
            print(f"✅ Imported {len(objects)} objects into {model_label}")
        except Exception as e:
            print(f"❌ Failed to import {model_label}: {e}")
